In [1]:
try:
    import crewai
    print("CrewAI is installed! Version:", crewai.__version__)
except ImportError:
    print("CrewAI not found. Please install it using the command above.")

CrewAI is installed! Version: 0.102.0


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
import requests

city_map = {
    1: "Seabrook City",
    2: "Highland Park City",
    3: "Baytown City",
    4: "Ridgeview City",
    5: "Shoreline City"
}

In [4]:
from crewai.tools import tool

@tool
def resourcing(number: int) -> str:
    """
    Calculates the amount of resources needed based on the population.

    Args:
        number (int): The number of people.

    Returns:
        str: A string describing the required quantities of apples, bananas, and oranges.
    """
    return f"{number * 3} apples, {number * 2} bananas, and {number} oranges are needed for {number} people."

@tool
def helping(number: int) -> str:
    """
    Determines the number of resources dispatched for disaster response.

    Args:
        number (int): The total number of people hurt in the disaster.

    Returns:
        str: A detailed message indicating the resources allocated, 
             including helicopters, police personnel, and special forces units.
    """
    return f"{number//100} helicopters dispatched, {number//50} police dispatched and {number//200} special forces"


In [5]:
from crewai import Agent, LLM
from crewai_tools import RagTool, DOCXSearchTool
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatLiteLLM
import os
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import pathlib
import pyttsx3 

llm = LLM(
    model='gemini/gemini-2.0-flash',  
    temperature=0.5,
    api_key=os.getenv("GOOGLE_API_KEY"),  
)

tool = DOCXSearchTool(
    config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model='gemini/gemini-2.0-flash',
                temperature=0.5,
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/text-embedding-004",
                task_type="retrieval_document",
                title="Embeddings"
            ),
        ),
    )
)

Analysed_Site = city_map[1]

data_collector = Agent(
    role="Document Searcher ",
    goal=f'Collect All the data available about {Analysed_Site} city in DOCX files. For example(just for reference, read through document for actual data)',
    backstory = "You're a proactive data collector agent who collects all the data available about the city in DOCX files. You will use the tool provided to search through documents and gather relevant information.",
    llm=llm,
    verbose=True,
    tools=[tool],
)

needs_analyst = Agent(
    role="Needs Analyser",
    goal=f"Based on the information available from data_collector agent, generate a proffessional grade News report for only {Analysed_Site} city about TOTAL POPULATION and Essential Needs for the city people",
    backstory="You’re a News reporter and analyst who finds out the needs of population",
    llm=llm,
    verbose=True
)

help_dispatcher = Agent(
    role="Aid Dispatcher",
    goal= f"Based on that Number of People Hurt, Number of People Stuck and where they are stuck from the information gathered by data_collector about the city, generate a proffessional grade News report for only {Analysed_Site} city about number of helicopters dispatched, police dispatched, special forces dispatched using the tool you are provided with. ",
    backstory="You’re a proactive News report generator agent who provides the numbers of the help that is dispatched effectively to meet the needs of the number of people hurt due to disaster.",
    llm=llm,
    verbose=True,
    tools = [helping]
    
)
 
resource_allocator = Agent(
    role="Resources Allocator",
    goal= f"Based on that Number of Population in the input city information provided by data_collector for the city, generate a proffessional grade News report for only {Analysed_Site} city what is the population of city and use given tool to estimate amount of foods needed returned by the tool you are provided with ",
    backstory="You’re a proactive News report generator agent who generates information about resources that needs to be dispatched effectively to meet the needs of the population.",
    llm=llm,
    verbose=True,
    tools = [resourcing]  
)

damage_analyser = Agent(
    role="Damage Analyser",
    goal= f"Based on the Damaged Infrastructure information collected by data_collector, generate report for only {Analysed_Site} city of the Damaged Infrastructure in the disaster-affected area for the input city and identify critical zones requiring immediate attention.",
    backstory="You’re a proactive News report generator agent who generates information about disaster impacts to provide Infrastructure damage.",
    llm=llm,
    verbose=True
)

print("Agents created successfully!")

d:\All Projects\Personal\Post Disaster Ai Agents Backend\.venv\Lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "ChangeTrackingData" shadows an attribute in parent "BaseModel"
  warnings.warn(
d:\All Projects\Personal\Post Disaster Ai Agents Backend\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


Agents created successfully!


d:\All Projects\Personal\Post Disaster Ai Agents Backend\.venv\Lib\site-packages\alembic\config.py:564: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [6]:
from crewai import Task, Crew
import os
from pathlib import Path

dataCollect_task = Task(
    description=f'Collect All the data available about {Analysed_Site} city in DOCX files. For example(just for reference, read through document for actual data), you will find docx file in "Cities.docx"',
    expected_output="A properly formatted JSON containing all collected data about the city including population, disaster information, casualties, and infrastructure damage",
    agent=data_collector,
)

needs_task = Task(
    description=f"Based on the information available from data_collector agent, generate a proffessional grade News report for only {Analysed_Site} city of what is the TOTAL POPULATION and Essential Needs for the city people",
    expected_output="A Proffessional grade News report of what is the Essential Needs for the city people within 30-40 words, so only include report related to TOTAL POPULATION and essential needs of the city people",
    agent=needs_analyst,
    context = [dataCollect_task]
)

dispatch_task = Task(
    description= f"Based on that Number of People Hurt, Number of People Stuck and where they are stuck from the information gathered by data_collector about the city, generate a proffessional grade News report for only {Analysed_Site} city about number of helicopters dispatched, police dispatched, special forces dispatched using the tool you are provided with.",
    expected_output="A Proffessional grade News report of information and telling that the number of help has been dispatched in 30-40 words, so only include report related to help dispatched",
    agent= help_dispatcher,
    context = [dataCollect_task]
)

resource_allocation_task = Task(
    description= f"Based on that Number of Population in the input city information provided by data_collector for the city, generate a proffessional grade News report for only {Analysed_Site} city of what is the population of city and use given tool to estimate amount of foods needed returned by the tool you are provided with.",
    expected_output="A Proffessional grade News report of the amount of resources needed in that area in 30-40 words, so only include report related to resources needed",
    agent=resource_allocator,
    context=[dataCollect_task]
)

damageAnalysis_task = Task(
    description = f"Based on the Damaged Infrastructure information collected by data_collector, generate report of the Damaged Infrastructure in the disaster-affected area for only {Analysed_Site} city and identify critical zones requiring immediate attention.",
    expected_output="A Proffessional grade News report on the extent of damage, including severity levels and affected zones in 30-40 words, so only include report related to damage analysis",
    agent=damage_analyser,
    context=[dataCollect_task]
)

crew = Crew(
    agents=[data_collector, needs_analyst, help_dispatcher, resource_allocator, damage_analyser],
    tasks=[dataCollect_task, needs_task, dispatch_task, resource_allocation_task, damageAnalysis_task],
    verbose=True
)
print("Tasks and crew set up successfully!")

Tasks and crew set up successfully!


In [7]:
result = crew.kickoff()

agent_outputs = {}

for i, task_output in enumerate(result.tasks_output):
    agent_outputs[f"Agent{i}"] = task_output.raw

print("=== RAW OUTPUTS ONLY ===")
for agent_name, raw_output in agent_outputs.items():
    print(f"\n{agent_name}:")
    print(raw_output)
    print("-" * 50)

final_output = result.raw
print(f"\n=== FINAL RAW OUTPUT ===")
print(final_output)



# Agent: Document Searcher 
## Task: Collect All the data available about Seabrook City city in DOCX files. For example(just for reference, read through document for actual data), you will find docx file in "Cities.docx"


Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]2025-06-11 00:19:51,869 - 14764 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--1842d7920de90159c62612bb44cd421e882e375f90d2ae13929a6e4e9daa3d43
2025-06-11 00:19:51,870 - 14764 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--1842d7920de90159c62612bb44cd421e882e375f90d2ae13929a6e4e9daa3d43
2025-06-11 00:19:51,871 - 14764 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--1842d7920de90159c62612bb44cd421e882e375f90d2ae13929a6e4e9daa3d43
2025-06-11 00:19:51,872 - 14764 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--606ab97fee05891ec800a0608d039a864c76f2b85e573c0d9f34d1985d4220af
2025-06-11 00:19:51,873 - 14764 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of e



# Agent: Document Searcher 
## Thought: Okay, I need to collect all the data available about Seabrook City in the "Cities.docx" file. I will use the "Search a DOCX's content" tool to search for "Seabrook City" within the document and extract all relevant information.
## Using tool: Search a DOCX's content
## Tool Input: 
"{\"docx\": \"Cities.docx\", \"search_query\": \"Seabrook City\"}"
## Tool Output: 
Relevant Content:
Seabrook City

**Disaster**: Seabrook City has been severely impacted by a tsunami disaster.

**Population**: Seabrook City has a total population of 1,200,000 residents.

**Casualties**: Current casualty reports indicate 4,500 people have been injured in Seabrook City.

**Trapped Locations**:

- **Oceanfront Plaza**: 150 people in Seabrook City are currently trapped.

- **Skyline Shop No. 11**: 200 people in Seabrook City are currently trapped.

- **Wavecrest Apartments**: 350 people in Seabrook City are currently trapped.

**Infrastructure Damage**:

- **Oceanview 

In [8]:
agent_outputs.pop("Agent0", None)  # Remove the data collector's output if not needed
agent_outputs


{'Agent1': 'Seabrook City is grappling with the aftermath of the tsunami. The city, with a total population of 1,200,000, urgently requires food supplies, clothing, clean water, and medical supplies to address the immediate needs of its residents.',
 'Agent2': 'Seabrook City: In response to the Tsunami, 45 helicopters, 90 police personnel, and 22 special forces units have been dispatched to aid in rescue and relief efforts. Focus is on Oceanfront Plaza, Skyline Shop No. 11, and Wavecrest Apartments.',
 'Agent3': 'Seabrook City, with a population of 1,200,000, urgently requires substantial food supplies following the tsunami. An estimated 3,600,000 apples, 2,400,000 bananas, and 1,200,000 oranges are needed to address the immediate needs of the affected population.',
 'Agent4': "Seabrook City's infrastructure faces severe damage after the Tsunami. Oceanview Tower suffered complete structural collapse, and Eastside Pier is severely damaged, rendering it unsafe. Oceanfront Plaza, Skyline 

In [9]:
agent_outputs['Needs Analyst Agent'] = agent_outputs.pop('Agent1')
agent_outputs['Help Dispatcher Agent'] = agent_outputs.pop('Agent2')
agent_outputs['Resource Allocator Agent'] = agent_outputs.pop('Agent3')
agent_outputs['Damage Analyser'] = agent_outputs.pop('Agent4')
agent_outputs

{'Needs Analyst Agent': 'Seabrook City is grappling with the aftermath of the tsunami. The city, with a total population of 1,200,000, urgently requires food supplies, clothing, clean water, and medical supplies to address the immediate needs of its residents.',
 'Help Dispatcher Agent': 'Seabrook City: In response to the Tsunami, 45 helicopters, 90 police personnel, and 22 special forces units have been dispatched to aid in rescue and relief efforts. Focus is on Oceanfront Plaza, Skyline Shop No. 11, and Wavecrest Apartments.',
 'Resource Allocator Agent': 'Seabrook City, with a population of 1,200,000, urgently requires substantial food supplies following the tsunami. An estimated 3,600,000 apples, 2,400,000 bananas, and 1,200,000 oranges are needed to address the immediate needs of the affected population.',
 'Damage Analyser': "Seabrook City's infrastructure faces severe damage after the Tsunami. Oceanview Tower suffered complete structural collapse, and Eastside Pier is severely d

In [10]:
import json
json_string = json.dumps(agent_outputs, indent=4)  #converting to json incase need to send to frontend
print("JSON String (Serialized):")
print(json_string)

JSON String (Serialized):
{
    "Needs Analyst Agent": "Seabrook City is grappling with the aftermath of the tsunami. The city, with a total population of 1,200,000, urgently requires food supplies, clothing, clean water, and medical supplies to address the immediate needs of its residents.",
    "Help Dispatcher Agent": "Seabrook City: In response to the Tsunami, 45 helicopters, 90 police personnel, and 22 special forces units have been dispatched to aid in rescue and relief efforts. Focus is on Oceanfront Plaza, Skyline Shop No. 11, and Wavecrest Apartments.",
    "Resource Allocator Agent": "Seabrook City, with a population of 1,200,000, urgently requires substantial food supplies following the tsunami. An estimated 3,600,000 apples, 2,400,000 bananas, and 1,200,000 oranges are needed to address the immediate needs of the affected population.",
    "Damage Analyser": "Seabrook City's infrastructure faces severe damage after the Tsunami. Oceanview Tower suffered complete structural c